# 🏥 The Grumpy Doctogres: Advanced Fine-Tuning Pipeline

## Key Features implemented:
1.  **AutoAugment**: "The Storm" designed by the machine (as per Prof's advice).
2.  **Custom Architecture**: We don't just use `timm` directly. We wrap it in a custom `nn.Module` with a sophisticated classification head (Linear -> BN -> ReLU -> Dropout -> Linear).
3.  **Two-Stage Training**: 
    *   **Phase 1**: Frozen Backbone (Train only the head).
    *   **Phase 2**: Unfrozen Backbone (Fine-tune the whole network with low LR).
4.  **Class Balancing**: Weighted Loss to handle the imbalance between 'Luminal' and 'Triple Negative'.
5.  **Mixed Precision**: For faster training on RTX cards.

In [1]:
import os
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight

# --- REPRODUCIBILITY ---
SEED = 42
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


## 1. Data Loading & Splits

In [2]:
# --- CONFIGURATION ---
DATASET_ROOT = os.path.join(os.path.pardir, "an2dl2526c2") # Adjust path if needed
LABELS_CSV = os.path.join(DATASET_ROOT, "train_labels.csv")
CLEAN_DATA_DIR = os.path.join(DATASET_ROOT, "train_masked_noshreks")
TEST_DATA_DIR = os.path.join(DATASET_ROOT, "test_masked_noshreks")

IMG_SIZE = 300 # EfficientNet B3 works well with 300x300
BATCH_SIZE = 16 # Adjust based on VRAM (16 for 8GB, 32 for 16GB)

# --- LOAD DATA ---
labels_df = pd.read_csv(LABELS_CSV)
# Create dictionary mapping filename -> label
labels_map = dict(zip(labels_df.iloc[:, 0], labels_df.iloc[:, 1]))

# Filter filenames ensuring they exist in the clean folder
valid_filenames = sorted([f for f in os.listdir(CLEAN_DATA_DIR) if f in labels_map])
y_labels_raw = [labels_map[f] for f in valid_filenames]

# Encode Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_labels_raw)
CLASSES = label_encoder.classes_
NUM_CLASSES = len(CLASSES)

print(f"Classes: {CLASSES}")

# --- STRATIFIED SPLIT ---
X_train, X_temp, y_train, y_temp = train_test_split(
    valid_filenames, y_encoded, test_size=0.2, stratify=y_encoded, random_state=SEED
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=SEED
)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

# --- COMPUTE CLASS WEIGHTS ---
# This handles the imbalance (e.g., fewer Triple Negative samples)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"Class Weights: {class_weights}")

Classes: ['HER2(+)' 'Luminal A' 'Luminal B' 'Triple negative']
Train: 1009 | Val: 126 | Test: 127
Class Weights: [0.98151751 0.93081181 0.70856742 2.018     ]


## 2. Dataset & AutoAugment
Here we implement the Professor's suggestion: **AutoAugment**.

In [3]:
class HistologyDataset(Dataset):
    def __init__(self, filenames, labels, root_dir, transform=None):
        self.filenames = filenames
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.filenames[idx])
        img = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

# --- ⚡ AUTO AUGMENTATION ⚡ ---
# The "Machine chooses the storm" approach
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    # AutoAugment: Learned augmentation policy from ImageNet
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- DATALOADERS ---
# num_workers=0 is safer on Windows to avoid "broken pipe" errors
train_dataset = HistologyDataset(X_train, y_train, CLEAN_DATA_DIR, train_transform)
val_dataset = HistologyDataset(X_val, y_val, CLEAN_DATA_DIR, val_transform)
test_dataset = HistologyDataset(X_test, y_test, CLEAN_DATA_DIR, val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

## 3. Custom Model Architecture
We use EfficientNet B3 as a backbone, but we define a custom **Classification Head**. We also add a method to freeze/unfreeze the backbone for 2-stage training.

In [4]:
class RefinedEfficientNet(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(RefinedEfficientNet, self).__init__()
        
        # 1. Load Backbone (EfficientNet B3)
        # num_classes=0 removes the default classifier
        self.backbone = timm.create_model('efficientnet_b3', pretrained=pretrained, num_classes=0)
        
        # Get the input dimension of the classification head (usually 1536 for B3)
        in_features = self.backbone.num_features
        
        # 2. Custom Head
        # More complex than a single layer: [Dense -> BN -> ReLU -> Drop -> Dense]
        self.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),  # Dropout to reduce overfitting
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.backbone(x)
        output = self.head(features)
        return output
    
    def freeze_backbone(self):
        # Freezes the feature extractor, allowing only the head to learn
        for param in self.backbone.parameters():
            param.requires_grad = False
        print("Backbone frozen.")
            
    def unfreeze_backbone(self):
        # Unfreezes everything for fine-tuning
        for param in self.backbone.parameters():
            param.requires_grad = True
        print("Backbone unfrozen.")

model = RefinedEfficientNet(num_classes=NUM_CLASSES).to(device)

## 4. Training Engine
Standard loop with Mixed Precision (`scaler`) and Early Stopping.

In [5]:
def train_one_epoch(model, loader, criterion, optimizer, scaler):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Mixed Precision Forward
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # Mixed Precision Backward
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
    return running_loss / total, correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    return running_loss / total, correct / total

def run_training(model, epochs, patience, save_name, lr):
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)
    # Scheduler: Drop LR if validation loss stalls
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    scaler = torch.cuda.amp.GradScaler()
    
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, scaler)
        val_loss, val_acc = validate(model, val_loader, criterion)
        
        scheduler.step(val_loss)
        
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_name)
            print(f"--> Saved Best Model ({best_val_acc:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print("Early Stopping!")
            break
            
    # Load best model before returning
    model.load_state_dict(torch.load(save_name))
    return model

## 5. Phase 1: Warmup (Head Only)
We freeze the backbone so gradients don't destroy pre-trained weights. We use a higher LR (1e-3) to get the head into shape.

In [6]:
print("=== PHASE 1: WARMUP ==")
model.freeze_backbone()
# Short training (10 epochs) just for the head
model = run_training(model, epochs=10, patience=5, save_name="warmup_checkpoint.pth", lr=1e-3)

e:\miniconda3\envs\an2dl-kaggle\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_38196\1767922644.py:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


=== PHASE 1: WARMUP ==
Backbone frozen.


Training:   0%|          | 0/63 [00:00<?, ?it/s]

C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_38196\1767922644.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/10 - Train Loss: 1.6175 Acc: 0.2440 | Val Loss: 1.4526 Acc: 0.3492
--> Saved Best Model (0.3492)


...

Epoch 3/10 - Train Loss: 1.4353 Acc: 0.3175 | Val Loss: 1.4383 Acc: 0.2698


Epoch 4/10 - Train Loss: 1.4231 Acc: 0.2996 | Val Loss: 1.4607 Acc: 0.2619


Epoch 5/10 - Train Loss: 1.3835 Acc: 0.3333 | Val Loss: 1.4233 Acc: 0.3016


Epoch 6/10 - Train Loss: 1.3803 Acc: 0.3433 | Val Loss: 1.4546 Acc: 0.2937
Early Stopping!


C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_38196\1767922644.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_name))


## 6. Phase 2: Fine-Tuning (Full Network)
We unfreeze the backbone and train everything with a **low Learning Rate** (1e-4) to gently adapt the feature extractor.

In [7]:
print("\n=== PHASE 2: FINE-TUNING ==")
model.unfreeze_backbone()
# Longer training with lower LR
model = run_training(model, epochs=50, patience=10, save_name="final_best_model.pth", lr=1e-4)


=== PHASE 2: FINE-TUNING ==
Backbone unfrozen.


C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_38196\1767922644.py:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Training:   0%|          | 0/63 [00:00<?, ?it/s]C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_38196\1767922644.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/50 - Train Loss: 1.4536 Acc: 0.3026 | Val Loss: 1.4252 Acc: 0.3333
--> Saved Best Model (0.3333)


Epoch 2/50 - Train Loss: 1.4032 Acc: 0.3046 | Val Loss: 1.4307 Acc: 0.3571
--> Saved Best Model (0.3571)


Epoch 3/50 - Train Loss: 1.3307 Acc: 0.3274 | Val Loss: 1.4151 Acc: 0.3016


...

Epoch 19/50 - Train Loss: 0.5434 Acc: 0.7887 | Val Loss: 1.6256 Acc: 0.3492


Epoch 20/50 - Train Loss: 0.5610 Acc: 0.7986 | Val Loss: 1.6051 Acc: 0.3651


Epoch 21/50 - Train Loss: 0.5349 Acc: 0.7996 | Val Loss: 1.5542 Acc: 0.3810


Epoch 22/50 - Train Loss: 0.5141 Acc: 0.8115 | Val Loss: 1.5648 Acc: 0.3651
Early Stopping!


C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_38196\1767922644.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_name))


## 7. Final Evaluation on Test Split (Labeled)

In [8]:
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

print("\nClassification Report (Internal Test Set):")
print(classification_report(all_targets, all_preds, target_names=CLASSES))


Classification Report (Internal Test Set):
                 precision    recall  f1-score   support

        HER2(+)       0.31      0.27      0.29        33
      Luminal A       0.19      0.24      0.21        34
      Luminal B       0.43      0.42      0.43        45
Triple negative       0.42      0.33      0.37        15

       accuracy                           0.32       127
      macro avg       0.34      0.32      0.32       127
   weighted avg       0.33      0.32      0.33       127



## 8. Generate Submission (Unlabeled Data)

In [9]:
class SubmissionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.filenames = sorted([f for f in os.listdir(root_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        fname = self.filenames[idx]
        img_path = os.path.join(self.root_dir, fname)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, fname

if os.path.exists(TEST_DATA_DIR):
    sub_dataset = SubmissionDataset(TEST_DATA_DIR, transform=val_transform)
    sub_loader = DataLoader(sub_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    sub_preds = []
    sub_files = []

    model.eval()
    with torch.no_grad():
        for images, filenames in tqdm(sub_loader, desc="Inferring"):
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            sub_preds.extend(preds.cpu().numpy())
            sub_files.extend(filenames)

    # Convert indices back to strings
    pred_labels = label_encoder.inverse_transform(sub_preds)

    df_sub = pd.DataFrame({'sample_index': sub_files, 'label': pred_labels})
    df_sub.to_csv("submission.csv", index=False)
    print("\n✅ Submission.csv generated!")
    print(df_sub.head())
else:
    print("Test data folder not found. Skipping submission generation.")

Inferring: 100%|██████████| 60/60 [00:07<00:00,  7.63it/s]


✅ Submission.csv generated!
   sample_index      label
0  img_0000.png  Luminal A
1  img_0001.png  Luminal A
2  img_0002.png  Luminal A
3  img_0003.png    HER2(+)
4  img_0004.png  Luminal A
